In [ ]:
%load_ext autoreload
%autoreload 2

%store -r the_page
%store -r the_editor

# Basic Editor Info from Wikipedia

In [ ]:
# The commented code, replicates the query
# from external.wikipedia import WikipediaDV, WikipediaAPI
# wikipedia_dv = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))
# the_editor = wikipedia_dv.get_editor(the_editor['name'])

the_editor.to_frame('value')

# Modified Pages per Editor

In [ ]:
from external.xtools import XtoolsAPI, XtoolsDV
xtools_api = XtoolsAPI(project = 'en.wikipedia.org')
xtools_dv = XtoolsDV(xtools_api)
editor_info = xtools_dv.get_modified_pages_counts_per_editor(the_editor['name'])
editor_info.to_frame('value')

# Created Pages per Editor

In [ ]:
from external.xtools import XtoolsAPI, XtoolsDV
xtools_api = XtoolsAPI(project = 'en.wikipedia.org')
xtools_dv = XtoolsDV(xtools_api)
created_pages = xtools_dv.get_created_pages_per_editor(the_editor['name'])
created_pages[['human_time','page_title', 'page_len', 'rev_id', 'rev_len', 'rev_timestamp']]

# Select page created by editor

In [ ]:
from external.wikipedia import WikipediaDV, WikipediaAPI
def page_fn(page):
    global the_page
    global page_inputname
    page_inputname = page
    
    wikipedia_dv = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))
    the_page = wikipedia_dv.get_page(page)
    %store the_page
       
    # display the data that will be passed to the next notebook
    display(the_page.to_frame('values'))

from IPython.display import display, Markdown as md
display(md('**Select a page based on the previous information**'))

from ipywidgets import interact
interact(page_fn, page=created_pages['page_title']);


In [ ]:
from IPython.display import HTML
HTML('<a href="1. General Metadata of a Wikipedia Article.ipynb" target="_blank">Go to next workbook</a>')